In [5]:
import torch
import sys
import os
os.chdir(os.path.dirname(os.path.realpath(".")) + '/FidelityFusion')

import GaussianProcess.kernel as kernel
from FidelityFusion_Models import *
from FidelityFusion_Models.MF_data import MultiFidelityDataManager
from DMF_acq import *

../


# Define a Simple Two fidelity objective function
This is a very simple two fidelity sample, named as 'Non Linear Sin' function.
\begin{aligned}
    f_{low}(x) &= \sin(8\pi x),\\
    f_{high}(x) &= (x - \sqrt{2})f_{low}(x)^2.\\
\end{aligned}

We use this objective function to generate a small size training data set, with 8 low-fidleity samples and 4 high-fidelity training samples under the non-subset assumption.

To be mentioned, the fidelity indicator starts from 0, which means 0 represents low-fidleity and 1 stands for high-fidleity.

In [6]:
def objective_function(x, s):
    xtr = x
    if s == 0:
        Ytr = torch.sin(xtr * 8 * torch.pi)
    else:
        Ytr_l = torch.sin(xtr * 8 * torch.pi)
        Ytr = (xtr - torch.sqrt(torch.ones(xtr.shape[0])*2).reshape(-1, 1)) * torch.pow(Ytr_l, 2)
    
    return Ytr

train_xl = torch.rand(8, 1) * 10
train_xh = torch.rand(4, 1) * 10
train_yl = objective_function(train_xl, 0)
train_yh = objective_function(train_xh, 1)

data_shape = [train_yl[0].shape, train_yh[0].shape]

initial_data = [
                    {'fidelity_indicator': 0,'raw_fidelity_name': '0', 'X': train_xl, 'Y': train_yl},
                    {'fidelity_indicator': 1, 'raw_fidelity_name': '1','X': train_xh, 'Y': train_yh},
                ]

# Initialize the surrogate model: ResGP

These part we use the simplest surrogate model ResGP as our MF model and train by the initiated training dataset. It can also be replaced by other MF model in FidelityFusion Models (i.e. AR, GAR).

In [7]:
fidelity_manager = MultiFidelityDataManager(initial_data)
kernel1 = kernel.SquaredExponentialKernel(length_scale = 1., signal_variance = 1.)
# model = AR(fidelity_num=2, kernel=kernel1, rho_init=1.0, if_nonsubset=True)
# train_AR(model, fidelity_manager, max_iter=100, lr_init=1e-3)
model = ResGP(fidelity_num=2, kernel=kernel1, if_nonsubset=True)
train_ResGP(model, fidelity_manager, max_iter=100, lr_init=1e-3)

fidelity: 0 iter 0 nll:10.55080
fidelity: 0 iter 1 nll:10.54580
fidelity: 0 iter 2 nll:10.54081
fidelity: 0 iter 3 nll:10.53581
fidelity: 0 iter 4 nll:10.53082
fidelity: 0 iter 5 nll:10.52583
fidelity: 0 iter 6 nll:10.52085
fidelity: 0 iter 7 nll:10.51587
fidelity: 0 iter 8 nll:10.51089
fidelity: 0 iter 9 nll:10.50591
fidelity: 0 iter 10 nll:10.50094
fidelity: 0 iter 11 nll:10.49597
fidelity: 0 iter 12 nll:10.49100
fidelity: 0 iter 13 nll:10.48604
fidelity: 0 iter 14 nll:10.48108
fidelity: 0 iter 15 nll:10.47612
fidelity: 0 iter 16 nll:10.47116
fidelity: 0 iter 17 nll:10.46622
fidelity: 0 iter 18 nll:10.46127
fidelity: 0 iter 19 nll:10.45633
fidelity: 0 iter 20 nll:10.45139
fidelity: 0 iter 21 nll:10.44645
fidelity: 0 iter 22 nll:10.44152
fidelity: 0 iter 23 nll:10.43659
fidelity: 0 iter 24 nll:10.43167
fidelity: 0 iter 25 nll:10.42675
fidelity: 0 iter 26 nll:10.42184
fidelity: 0 iter 27 nll:10.41693
fidelity: 0 iter 28 nll:10.41202
fidelity: 0 iter 29 nll:10.40712
fidelity: 0 iter 30 

# Define the mean and variance functions for acq function
The defined mean and variance functions extract the predictive mean and variance from the trained surrogate model (model) when provided with input points (x) and fidelity indicator (s). 
These functions are crucial components in the computation of acquisition functions, such as the Upper Confidence Bound (UCB), and are used to guide the selection of the next point for evaluation in the Bayesian optimization process.

In [8]:
def mean_function(x, s):
    mean, _ = model.forward(fidelity_manager, x, s)
    return mean.reshape(-1, 1)
    
def variance_function(x, s):
    _, variance = model.forward(fidelity_manager, x, s)
    return variance

# Initialize and optimize MF_acq function

In this section, we initiate a sample for MF discrete acquistion function. According to the different acq, there is a little different between the initiate parameter setting.
If want to use the KG/EI/PI acq, need specify the current best function (f_best) when initiate. But UCB do not need the f_best when initiating.

Then we use the Opitmizer to find the new_x and use different selection strategy to select next_s.

In [11]:
acq = DMF_UCB(mean_function, variance_function, 2, train_xl.shape[1], torch.ones(1).reshape(-1, 1))

# Use Opitmizer to find the new_x
new_x = optimize_acq_mf(fidelity_manager, acq, 10, 0.01) 
# Use different selection strategy to select next_s
new_s = acq.acq_selection_fidelity(gamma=[0.1, 0.1], new_x=new_x)
print(new_x, new_s)

iter 0 x: Parameter containing:
tensor([[0.4920]], requires_grad=True) Negative Acquisition Function: -0.1446552574634552
iter 1 x: Parameter containing:
tensor([[0.4824]], requires_grad=True) Negative Acquisition Function: -0.14496617019176483
iter 2 x: Parameter containing:
tensor([[0.4728]], requires_grad=True) Negative Acquisition Function: -0.14527581632137299
iter 3 x: Parameter containing:
tensor([[0.4632]], requires_grad=True) Negative Acquisition Function: -0.1455923616886139
iter 4 x: Parameter containing:
tensor([[0.4536]], requires_grad=True) Negative Acquisition Function: -0.1459185779094696
iter 5 x: Parameter containing:
tensor([[0.4439]], requires_grad=True) Negative Acquisition Function: -0.1462557166814804
iter 6 x: Parameter containing:
tensor([[0.4341]], requires_grad=True) Negative Acquisition Function: -0.14660455286502838
iter 7 x: Parameter containing:
tensor([[0.4241]], requires_grad=True) Negative Acquisition Function: -0.14696544408798218
iter 8 x: Parameter 